## Matrix Factorization

Matrix factorization is one of model-based algorithm (or Latent Factor Model). Using the nature of the matrix.

### Define

- Basic concept of matrix factorization is `matrix completion`.
- Fill the below image's `?` is main purpose of matrix completion.

<img src="img/mf.png" alt="MF" style="width: 300px;"/>

- Matrix's row, column is factor of concept (e.g. Row: user, Column: Movie, Value: Rating)
- And matrix factorization divide it two parts(P, Q). And they called `Latent Factor`.
    - $ R \approx P X Q^T = \hat{R} $
    - And Latent factor has their own dimension `k`.
    - $ \hat{r_{ij}} = p_i^T q_j = \sum_{k=1}^k p_{ik}q_{kj} $
    - The example of using latent factor
        - **e.g.** When we have (User, Movie, Rating) matrix, we implicitically knows about every user has prefered genre. In this situation, user latent factor's dimension(k) maybe works like `prefer genre`. if k=3, each dimension value represent preference score of each genre. (romance = -1, action = 2, comedy = 10)

----
### Cost function & Train

- Cost function of Matrix Factorization is below.

$$ Cost = \sum_{i,u \in R_{train}} (r(i,u) - \hat{r}(i,u) )^2 + \lambda(b(i)^2 + b(u)^2 + ||p(i)||^2 ||q(u)||^2) $$

- And we can use Gradient Descent.

$$ \grave{p_{ik}} = p_{ik} +\alpha \frac{dCost}{dp}$$


$$ \grave{q_{kj}} = q_{kj} +\alpha \frac{dCost}{dq}$$

- Here is my process of deviation

<img src="img/process1.png" alt="MF" style="width: 300px;"/>
<img src="img/process2.png" alt="MF" style="width: 300px;"/>

In [32]:
import numpy as np

# 7X5 Matrix
R = np.array([
        [1, 0, 0, 1, 3],
        [2, 0, 3, 1, 1],
        [1, 2, 0, 5, 0],
        [1, 0, 0, 4, 4],
        [2, 1, 5, 4, 0],
        [5, 1, 5, 4, 0],
        [0, 0, 0, 1, 0],
    ])

# P, Q is (7 X k), (k X 5) matrix
P = np.random.normal(size=(R.shape[0], 4))
Q = np.random.normal(size=(R.shape[1], 4))

# b_P, b_Q is (7 X 1), (5 X 1)
b_P = np.ones(R.shape[0])
b_Q = np.ones(R.shape[1])
b = np.mean(R[np.where(R != 0)])

In [33]:
P

array([[ 1.14544264,  0.76617751, -0.15163809, -1.14918166],
       [-1.19423342,  1.62504796,  0.14128756,  0.07143081],
       [-1.48406439, -1.2017517 ,  0.5767384 ,  0.49059089],
       [-0.76688018, -1.12450154,  1.19667241, -2.22018095],
       [ 0.63085357,  0.32312185,  0.81577999,  1.01556337],
       [ 0.87686163,  0.60873574,  0.7956862 , -0.82178764],
       [-0.41426823,  0.78840721, -0.67253972, -0.68151125]])

In [34]:
Q

array([[-0.24450083, -0.56573807, -0.38928391,  0.49186013],
       [-0.23816813, -0.73399454,  1.10309967,  0.75727339],
       [ 0.68286032,  2.53800163,  0.0578524 ,  0.75766522],
       [ 0.38358876, -0.82099596,  1.36663131,  0.09294516],
       [ 0.35038026,  1.91152701,  0.09604263, -0.52097559]])

In [47]:
R_hat = b + b_P[:, np.newaxis] + b_Q[np.newaxis:, ] + P.dot(Q.T)
R_hat

array([[ 3.37118526,  2.71821442,  6.4381786 ,  4.08721513,  7.04095046],
       [ 3.94368164,  3.89250802,  7.96208329,  2.99838392,  7.25515227],
       [ 5.65042787,  6.83415649,  0.93252199,  5.84205877,  1.57354699],
       [ 3.85672146,  5.23769848, -0.39967323,  6.64900861,  3.44428597],
       [ 4.43580764,  5.87243597,  6.65842961,  5.77687787,  4.97886979],
       [ 3.3181783 ,  4.19066317,  6.15804784,  5.43852227,  6.56630999],
       [ 4.17276672,  2.85291931,  5.75373514,  2.80226431,  6.24327761]])

## Alternating Least Squares (with Implicit feedback)
----

ALS is specific optimizer for Matrix Factorization.

- Learn `User Latent Factor` and `Item Latent Factor` alternately.
- Because training both factors sometimes very inefficient.
- There are two step in ALS.
    - Set `Item Latent Factor` as constant for learn to `User Latent Factor`
- The original cost function of matrix factorization is below.

$$ min_{q,p} \sum_{u,i}(r_{ui}-q_i^Tp_u)^2 + \lambda(||q_i||^2 + ||p_u||^2) $$

- And there is ALS's cost function.

$$ min_{x,y} \sum c_{ui}(p_{ui}-x_i^Ty_u)^2 + \lambda(||x_i||^2 + ||y_u||^2) $$

- We can see two difference like this
    - $ p_{ui} $ is preference score(binary score) in Explicit feedback
        - p is 1 if $ r_{ui} > 0 $
        - p is 0 if $ r_{ui} = 0 $
    - $ c_{ui} $ is confidence for $ p_{ui} $.
        - $ c_{ui} = 1 + \alpha* r_{ui} $
        - If there are no score(p score), sparse matrix is very inefficient. (Almost implicit situation has sparse matrix problem)
        - c score can solve this problem. If we use this, many zero values turn to non-zero (almost zero) value. So the model's performance is better than before.

- Learning Step
    - 1) Fix Y's latent factor - $ \frac{dL(x_i)}{dx_i} = -2\sum c_{ui}(p_{ui}-x_i^Ty_u)*y_u + 2\lambda x_i $
        - Find $ \frac{dL(x_i)}{dx_i} $ is 0
        - $ \sum_u c_{iu}(x_i^Ty_u)*y_u +\lambda x_i = \sum_u c_{iu}(p_{iu})*y_u $
        - $ \sum_u c_{iu}(y_u^Tx_i)*y_u +\lambda x_i = \sum_u c_{iu}(p_{iu})*y_u $
        - $ (\sum_u c_{iu}*y_u*y_u^T +\lambda I )x_i = \sum_u c_{iu}(p_{iu})*y_u $
        - $ x_i = (Y^T C^i Y + \lambda I)^{-1} Y^T C^i p(i) $
    - 2) Fix X's latent factor
    - iter loop~~

----
#### references
- https://yamalab.tistory.com/92
- http://yifanhu.net/PUB/cf.pdf
- https://yeomko.tistory.com/4
- https://datascienceschool.net/view-notebook/fcd3550f11ac4537acec8d18136f2066/